In [1]:
import os
import sys
import re
import csv

import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt

In [2]:
# list shp files recursively
os.chdir('..')
abs_path = os.getcwd()

HIFLD_path = os.path.join(abs_path, 'data/HIFLD')
shp_files = [os.path.join(root, name) \
             for root, dirs, files in os.walk(HIFLD_path) \
             for name in files \
             if name.endswith(('.shp'))]

In [3]:
for file in shp_files:
    with open(file, 'rb') as file:
        row_count = len(file.readlines()) - 1
        print('There are {} observations in {}'.format(row_count, file))

There are 267622 observations in <_io.BufferedReader name='C:\\Users\\bchan\\OneDrive - UW\\CLAD\\CLAD_Geospatial\\data/HIFLD\\All_Places_Of_Worship\\AllPlacesOfWorship.shp'>
There are 93348 observations in <_io.BufferedReader name='C:\\Users\\bchan\\OneDrive - UW\\CLAD\\CLAD_Geospatial\\data/HIFLD\\FDIC_Insured_Banks\\FDIC_Insured_Banks.shp'>
There are 54785 observations in <_io.BufferedReader name='C:\\Users\\bchan\\OneDrive - UW\\CLAD\\CLAD_Geospatial\\data/HIFLD\\Fire_Stations\\Fire_Stations.shp'>
There are 5056 observations in <_io.BufferedReader name='C:\\Users\\bchan\\OneDrive - UW\\CLAD\\CLAD_Geospatial\\data/HIFLD\\Prison_Boundaries\\Prison_Boundaries.shp'>
There are 112783 observations in <_io.BufferedReader name='C:\\Users\\bchan\\OneDrive - UW\\CLAD\\CLAD_Geospatial\\data/HIFLD\\Public_Schools\\PublicSchools.shp'>
There are 5298 observations in <_io.BufferedReader name='C:\\Users\\bchan\\OneDrive - UW\\CLAD\\CLAD_Geospatial\\data/HIFLD\\Urgent_Care_Facilities\\UrgentCareFac

In [17]:
FDIC_shapes = gpd.read_file(FDIC_file, rows=100)
FDIC_shapes.columns

Index(['OBJECTID', 'ADDRESBR', 'BRNUM', 'BRSERTYP', 'CBSABR', 'CBSANAMB',
       'CITYBR', 'CNTRYNAB', 'CNTYNAMB', 'DEPSUMBR', 'GEOCODE_CE', 'NAMEBR',
       'STALPBR', 'STCNTYBR', 'STNAMEBR', 'UNINUMBR', 'ZIPBR', 'CERT',
       'ADDRESS', 'ASSET', 'BKCLASS', 'CITY', 'CNTRYNA', 'DENOVO', 'DEPDOM',
       'NAMEFULL', 'NAMEHCR', 'REGAGNT', 'REPDTE', 'RSSDID', 'STALP', 'STCNTY',
       'STNAME', 'ZIP', 'BKMO', 'LOC_NAME', 'STATUS', 'SCORE', 'x', 'y',
       'GeocodeSou', 'STD_ADDR_B', 'STD_ADDR', 'ZIP4BR', 'geometry'],
      dtype='object')

In [31]:
FDIC_shapes.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [16]:
Prison_shapes = gpd.read_file(Prison_file, rows=100)
Prison_shapes.columns

Index(['FID', 'FACILITYID', 'NAME', 'ADDRESS', 'CITY', 'STATE', 'ZIP', 'ZIP4',
       'TELEPHONE', 'TYPE', 'STATUS', 'POPULATION', 'COUNTY', 'COUNTYFIPS',
       'COUNTRY', 'NAICS_CODE', 'NAICS_DESC', 'SOURCE', 'SOURCEDATE',
       'VAL_METHOD', 'VAL_DATE', 'WEBSITE', 'SECURELVL', 'CAPACITY',
       'SHAPE_Leng', 'GlobalID', 'CreationDa', 'Creator', 'EditDate', 'Editor',
       'SHAPE_Le_1', 'SHAPE_Area', 'geometry'],
      dtype='object')

In [30]:
Prison_shapes.crs

<Projected CRS: EPSG:3857>
Name: WGS 84 / Pseudo-Mercator
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: World between 85.06°S and 85.06°N.
- bounds: (-180.0, -85.06, 180.0, 85.06)
Coordinate Operation:
- name: Popular Visualisation Pseudo-Mercator
- method: Popular Visualisation Pseudo Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [15]:
UrgentCare_shapes = gpd.read_file(UrgentCare_file, rows=100)
UrgentCare_shapes.columns

Index(['OBJECTID', 'ID', 'NAME', 'TELEPHONE', 'ADDRESS', 'ADDRESS2', 'CITY',
       'STATE', 'ZIP', 'ZIPP4', 'COUNTY', 'FIPS', 'DIRECTIONS', 'EMERGTITLE',
       'EMERGTEL', 'EMERGEXT', 'CONTDATE', 'CONTHOW', 'GEODATE', 'GEOHOW',
       'HSIPTHEMES', 'NAICSCODE', 'NAICSDESCR', 'GEOLINKID', 'X', 'Y',
       'ST_VENDOR', 'ST_VERSION', 'GEOPREC', 'PHONELOC', 'QC_QA', 'UCAOA_ID',
       'geometry'],
      dtype='object')

In [29]:
UrgentCare_shapes.crs

<Projected CRS: EPSG:3857>
Name: WGS 84 / Pseudo-Mercator
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: World between 85.06°S and 85.06°N.
- bounds: (-180.0, -85.06, 180.0, 85.06)
Coordinate Operation:
- name: Popular Visualisation Pseudo-Mercator
- method: Popular Visualisation Pseudo Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

# Full Address String

In [4]:
def read_shp(file, rows=100):
    df = gpd.read_file(file, rows=rows)

    return df

In [99]:
def str_address_match(str_address):
    match_result = re.match('\d+\s+\w+\s+\w+', str_address)

    return match_result

def state_match(state_str):
    states = ['IA', 'KS', 'UT', 'VA', 'NC', 'NE', 'SD', 'AL', 'ID', 'FM', 'DE', 'AK', 'CT', 'PR', 
              'NM', 'MS', 'PW', 'CO', 'NJ', 'FL', 'MN', 'VI', 'NV', 'AZ', 'WI', 'ND', 'PA', 'OK', 
              'KY', 'RI', 'NH', 'MO', 'ME', 'VT', 'GA', 'GU', 'AS', 'NY', 'CA', 'HI', 'IL', 'TN', 
              'MA', 'OH', 'MD', 'MI', 'WY', 'WA', 'OR', 'MH', 'SC', 'IN', 'LA', 'MP', 'DC', 'MT', 'AR', 'WV', 'TX']
    
    # match_result = re.compile(r'\b(' + '|'.join(state_str) + r')\b', re.IGNORECASE)
    match_result = re.search('\b(?i:AL|AK|AZ|AR|CA|CO|CT|DE|FL|GA|HI|ID|IL|IN|IA|KS|KY|LA|ME|MD|MA|MI|MN|MS|MO|MT|NV|NH|NJ|NM|NY|NC|ND|OH|OK|OR|PA|RI|SC|SD|TN|TX|UT|VT|VA|WA|WV|WI|WY|NE)\b', 
                            state_str)
    # match_result = re.match(r'\b[a-z]{2}\b', state_str, re.I)

    return match_result

In [5]:
def full_address(df):
    if 'ZIP' in df.columns:
        zip_col = 'ZIP'
    else:
        zip_col = 'ZIPCODE'

    if 'STATE' in df.columns:
        state_col = 'STATE'
    else:
        state_col = 'STNAME'

    # idx = np.where(df.columns.str.contains('ADDRESS'))
    # addr_col = df.columns[idx][0]

    if 'ADDRESS' in df.columns:
        addr_col = 'ADDRESS'
    elif 'ADDRESS' not in df.columns:
        addr_col = 'STREET'
    elif ('STREET' not in df.columns) and ('ADDRESS' not in df.columns):
        addr_col = 'STD_ADDR_B'

    # if ' ADDRESS' in df.columns:
        # names may make parsing addresses difficult
        # df['Full_Address'] = df[[addr_keep, 'CITY', state_keep]].agg(', '.join, axis=1) + ' ' + str(df['ZIP'])
        # df['Full_Address'] = df[['ADDRESS', 'CITY', state_col]].agg(', '.join, axis=1) + ' ' + df[zip_col].astype('str')
        # df['Place_Name'] = df['NAME']
        # df['Place_Name'] = df['NAME']
    # elif 'STREET' in df.columns:
       # df['Full_Address'] = df[['STREET', 'CITY', state_col]].agg(', '.join, axis=1) + ' ' + df[zip_col].astype('str')
    # else:
        # df['Full_Address'] = df[['ADDRESBR', 'CITY', state_col]].agg(', '.join, axis=1) + ' ' + df[zip_col].astype('str')
    df['Full_Address'] = df[[addr_col, 'CITY', state_col]].fillna('NaN').agg(', '.join, axis=1) + ' ' + df[zip_col].astype('str')
             
    return df

In [17]:
# for all shp_files
dict_address = dict()
for file in shp_files:
    basename = os.path.basename(file).split('/')[0]
    fname = os.path.basename(basename).split('.')[0]

    # print(file)
    df = read_shp(file, rows=2000)
    dict_address[fname] = full_address(df)
    # df_address[fname]['Facility_type'] = fname

# Calculate Lat/Lon

In [7]:
# convert to CRS 'EPSG:4326'
def convert_EPSG4326(dict):
    """
    dict from full_address
    maybe want to store source data before conversion
    """
    for fname in dict:
        dict[fname] = dict[fname].to_crs("EPSG:4326")

    return dict

In [8]:
def get_centroid(dict):
    """
    Returns Point object or a coordinate tuple (x, y)
    """
    for fname in dict:
        if 'x' in dict[fname].columns:
            dict[fname] = dict[fname].rename(columns={'x': 'source_lon', 'y': 'source_lat'})
            dict[fname]['Place_type'] = os.path.basename(fname)
        else:
            dict[fname]['source_centroid'] = dict[fname]['geometry'].centroid
            dict[fname]['source_lon'] = dict[fname]['geometry'].centroid.x
            dict[fname]['source_lat'] = dict[fname]['geometry'].centroid.y
            dict[fname]['Place_type'] = os.path.basename(fname)
        
    return dict

In [9]:
def keep_columns(dict):
    cols = ['Full_Address', 'Place_type', 'source_centroid', 'source_lon', 'source_lat']
    new_dict = {}
    for fname in dict:
        print(fname)
        new_dict[fname] = dict[fname][cols]

    return new_dict

In [10]:
def save_shp(dict, save_dir):
    for fname in dict:
        # dict[fname]['source_centroid'] = gpd.GeoSeries.from_wkt(dict[fname]['source_centroid'])
        shp_file = dict[fname].set_geometry('source_centroid')
        # shp_file = gpd.GeoDataFrame(dict[fname], geometry=dict[fname]['source_centroid'])
        # shp_file.to_file(os.path.join(save_dir, ('{}.shp'.format(fname))), driver='ESRI Shapefile')
        save_path = os.path.join(save_dir, f"{fname}")
        create_dir(save_path)
        shp_file.to_file(save_path, driver='ESRI Shapefile')

In [11]:
def create_dir(save_dir):
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

In [19]:
dict_EPSG4326 = convert_EPSG4326(dict_address)
dict_centroid = get_centroid(dict_EPSG4326)
final_dict = keep_columns(dict_centroid)

C:\Users\bchan\AppData\Local\Temp\ipykernel_27084\1302841443.py:10: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  dict[fname]['source_centroid'] = dict[fname]['geometry'].centroid
C:\Users\bchan\AppData\Local\Temp\ipykernel_27084\1302841443.py:11: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  dict[fname]['source_lon'] = dict[fname]['geometry'].centroid.x
C:\Users\bchan\AppData\Local\Temp\ipykernel_27084\1302841443.py:12: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  dict[fname]['source_lat'] = dict[fname]['geometry'].centroid.y
C:\Users\bchan\AppData\Local\Temp\ipykernel

AllPlacesOfWorship
FDIC_Insured_Banks
Fire_Stations
Prison_Boundaries
PublicSchools
UrgentCareFacs


In [21]:
# save files
save_dir = os.path.join(abs_path, 'output/HIFLD/centroids')
create_dir(save_dir)

save_shp(final_dict, save_dir)

C:\Users\bchan\AppData\Local\Temp\ipykernel_27084\471014251.py:9: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  shp_file.to_file(save_path, driver='ESRI Shapefile')
C:\Users\bchan\AppData\Local\Temp\ipykernel_27084\471014251.py:9: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  shp_file.to_file(save_path, driver='ESRI Shapefile')
C:\Users\bchan\AppData\Local\Temp\ipykernel_27084\471014251.py:9: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  shp_file.to_file(save_path, driver='ESRI Shapefile')
C:\Users\bchan\AppData\Local\Temp\ipykernel_27084\471014251.py:9: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  shp_file.to_file(save_path, driver='ESRI Shapefile')
C:\Users\bchan\AppData\Local\Temp\ipykernel_27084\471014251.py:9: UserWarning: Column names longer than 10 character